In [ ]:
# pip install x2paddle   # onnx ----> paddlepaddle

In [ ]:
# terminal  bash
# >>>x2paddle --framework=onnx --model=logs/model.onnx --save_dir=paddle_model
# 如果什么都没有发生，首先检查路径没有写错，然后就是环境的版本适配，
# numpy、tensorboard、protobuf等版本调整一下
# 同时也要注意paddlepaddle-gpu的版本， cuda11.3 ---- paddlepaddle-gpu 2.3.2
# paddle.utils.run_check()  并不是百分百的检测手段，最好还是拿其他的手段测试一下

In [ ]:
""" x2paddle的输出
paddle_model/
    +----/inference_model/
            +-----model.pdiparams
            +-----model.pdiparams.info
            +-----model.pdmodel
    +---- model.pdparams
    +---- x2paddle_code.py
"""

In [ ]:
# #error1#  paddle.to_tensor 会得到全0的tensor
# solution#当然是调整paddlepaddle-gpu的版本
# 如果要测试来得到问题所在的话：
# import paddle
# paddle.set_device('gpu:0')
# test_tensor = paddle.to_tensor([1.2, 2.2, 3.4], place=paddle.CUDAPlace(0))
# print("Test tensor stats:")
# print("Min:", test_tensor.min())  
# print("Max:", test_tensor.max())  
# print("Mean:", test_tensor.mean())  

In [ ]:
# #----------model load---------------------#
# import paddle
# model_path = 'paddle_model/inference_model/model'
# model = paddle.jit.load(model_path)

In [ ]:
# # input image-crop  640*640  PIL.Image.Image
# def paddle_preprocess(image, target_size=(640, 640), device='gpu'):
#     """将输入图像处理为模型可接受的格式。"""
#     image = image.resize(target_size, Image.BILINEAR)
#     image = np.array(image).astype('float32')
#     image = image.transpose((2, 0, 1))  # HWC to CHW
#     image = image / 255.0  # 归一化
#     # 明确地调用 CPU 或 GPU Place
#     image = paddle.to_tensor(image, place=paddle.CUDAPlace(0) if device == 'gpu' else paddle.CPUPlace())
#     image = image.unsqueeze(0)  # 增加batch维
#     # print("image shape", image.shape)  # image shape [1, 3, 640, 640]
#     # print("image type", type(image))  # image type <class 'paddle.Tensor'>
#     return image

In [ ]:
# def paddle_postprocess(pred, colors):
#     # print("pred type", pred.shape)  # pred type [1, 3, 640, 640]
#     # print("pred type", type(pred))  # pred type <class 'paddle.Tensor'>
#     """将分割结果转换为彩色图，并与原图混合。"""
#     pred = pred.squeeze(0)  # 移除batch维
#     pred = pred.argmax(0)   # 取概率最大的类别
#     pred = pred.numpy()
#     seg_img_colored = np.zeros((pred.shape[0], pred.shape[1], 3), dtype=np.uint8)
#     # print("seg_img_colored shape", seg_img_colored.shape)   # (640, 640, 3)
#     for c in range(len(colors)):
#         seg_img_colored[pred == c, :] = colors[c]
#     seg_img_colored = Image.fromarray(seg_img_colored)
#     return seg_img_colored

In [ ]:
# def paddle_seg_image(image, model, device='gpu'):
#     """使用 PaddlePaddle 进行分割推理并返回着色的分割结果图像。"""
#     # print("image size", image.size)  # image shape (640, 640)
#     # print("image type", type(image))  # image type <class 'PIL.Image.Image'>
#     old_image = image.copy()
#     # 定义每个类别的颜色
#     colors = [
#         [0, 0, 0],  # 背景 - 黑色
#         [255, 0, 0],  # 目标1 - 红色
#         [0, 255, 0],  # 目标2 - 绿色
#     ]
#     # 设置设备
#     paddle.set_device('gpu:0' if device == 'gpu' else 'cpu')
#     model.eval()
#     # 加载和预处理图像
#     input_data = paddle_preprocess(image, device=device)
#     # 执行模型推理
#     output = model(input_data)
#     # print("output shape", output.shape)  # output shape [1, 3, 640, 640]
#     # 处理输出并返回彩色分割图
#     seg_image = paddle_postprocess(output, colors)
#     # 将分割图与原图混合
#     original_image = old_image.convert('RGB').resize(seg_image.size)
#     blended_image = Image.blend(original_image, seg_image, 0.7)
#     return blended_image